In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

2023-03-05 10:21:05.028612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
__name__ = "RNN"

# Model

In [3]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [4]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [5]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [6]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [7]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

2023-03-05 10:21:11.649412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/400
11/11 [==============================] - 4s 28ms/step - loss: 1.5930 - categorical_accuracy: 0.2801
Epoch 2/400
11/11 [==============================] - 0s 30ms/step - loss: 1.5367 - categorical_accuracy: 0.2861
Epoch 3/400
11/11 [==============================] - 0s 30ms/step - loss: 1.4436 - categorical_accuracy: 0.4217
Epoch 4/400
11/11 [==============================] - 0s 30ms/step - loss: 1.5345 - categorical_accuracy: 0.4247
Epoch 5/400
11/11 [==============================] - 0s 32ms/step - loss: 1.7044 - categorical_accuracy: 0.4488
Epoch 6/400
11/11 [==============================] - 0s 33ms/step - loss: 11.5805 - categorical_accuracy: 0.2982
Epoch 7/400
11/11 [==============================] - 0s 32ms/step - loss: 27.4256 - categorical_accuracy: 0.2651
Epoch 8/400
11/11 [==============================] - 0s 36ms/step - loss: 26.3132 - categorical_accuracy: 0.2380
Epoch 9/400
11/11 [==============================] - 0s 30ms/step - loss: 26.6033 - categorical_accur

In [8]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 5)                 8

In [9]:
model.save(f"{modelPath}/{__name__}/test.h5")